In [1]:
from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env", verbose=True)

True

In [4]:
from tavily import TavilyClient
from langchain_openai import ChatOpenAI
import os

tavily = TavilyClient(api_key=os.getenv('TAVILY_API_KEY'))
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [5]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

# Add to vectorDB
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding = OpenAIEmbeddings(model="text-embedding-3-small")
)
retriever = vectorstore.as_retriever()

In [6]:
### Retrieval Grader
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

system = """You are a grader assessing relevance
    of a retrieved document to a user question. If the document contains keywords related to the user question,
    grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explanation.
    """

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "question: {question}\n\n document: {document} "),
    ]
)

retrieval_grader = prompt | llm | JsonOutputParser()
question = "What is prompt?"
docs = retriever.invoke(question)
doc_txt = docs[0].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

{'score': 'yes'}


In [11]:
### Generate

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

system = """You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
    Use three sentences maximum and keep the answer concise"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "question: {question}\n\n context: {context} "),
    ]
)

# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
question = "What is prompt?"
docs = retriever.invoke(question)
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

A prompt is a method used in prompt engineering, also known as in-context prompting, to communicate with language models (LLMs) to influence their behavior and achieve desired outcomes without altering the model's weights. This process involves experimentation and heuristics, as the effectiveness of prompts can vary significantly across different models. The primary goal of prompt engineering is to enhance alignment and steerability of the model.


In [13]:
### Hallucination Grader

system = """You are a grader assessing whether
    an answer is grounded in / supported by a set of facts. Give a binary 'yes' or 'no' score to indicate
    whether the answer is grounded in / supported by a set of facts. Provide the binary score as a JSON with a
    single key 'score' and no preamble or explanation."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "documents: {documents}\n\n answer: {generation} "),
    ]
)

hallucination_grader = prompt | llm | JsonOutputParser()
hallucination_grader.invoke({"documents": docs, "generation": generation})

{'score': 'yes'}

In [16]:
from typing import List
from typing_extensions import TypedDict

class GraphState(TypedDict):
    question: str
    generation: str
    web_search: bool
    documents: List[str]

In [14]:
from langchain_core.documents import Document

def docs_retrieval(state):
    print("1. DOCS RETRIEVE")
    question = state["question"]

    documents = retriever.invoke(question)
    return {"documents": documents, "question": question, "web_search": False}

def relevance_checker(state):
    print("2. RELEVANCE CHECKER")
    documents = state["documents"]
    question = state["question"]
    web_search = state["web_search"]

    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke({"question": question, "document": d.page_content})
        grade = score["score"]
        if grade.lower() == "yes":
            filtered_docs.append(d)
    return {"documents": filtered_docs, "question": question, "web_search": web_search}

def decide_to_generate(state):
    print("3. DECIDE TO GENERATE")
    documents = state["documents"]

    if len(documents) == 0:
        print("  USE WEB SEARCH")
        return "websearch"
    else:
        print("  GO TO GENERATE")
        return "generate"

def search_tavily(state):
    print("3-1. SEARCH TAVILY")
    question = state["question"]

    search_results = tavily.search(query=question, max_results=3)['results']
    documents = []
    for search_result in search_results:
        document = Document(
            page_content=search_result["content"],
            metadata={"source": search_result["url"], "title": search_result["title"]}
        )
        documents.append(document)
    return {"documents": documents, "question": question, "web_search": True}

def generate_answer(state):
    print("4. GENERATE ANSWER")
    question = state["question"]
    documents = state["documents"]

    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}

def hallucination_checker(state):
    print("5. HALLUCINATION CHECKER")
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = score["score"]

    if grade == "yes":
        print("  NO HALLUCINATION")
        return "success"
    else:
        print("  HALLUCINATION FOUND")
        return "failed"


In [17]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

workflow.add_node("docs_retrieval", docs_retrieval)
workflow.add_node("relevance_checker", relevance_checker)
workflow.add_node("generate_answer", generate_answer)
workflow.add_node("search_tavily", search_tavily)

workflow.set_entry_point("docs_retrieval")
workflow.add_edge("docs_retrieval", "relevance_checker")
workflow.add_conditional_edges(
    "relevance_checker",
    decide_to_generate,
    {"websearch": "search_tavily", "generate": "generate_answer"}
)
workflow.add_edge("search_tavily", "relevance_checker")
workflow.add_conditional_edges(
    "generate_answer",
    hallucination_checker,
    {"success": END, "failed": "generate_answer"}
)

In [28]:
app = workflow.compile()

inputs = {"question": "What is prompt?"}
for output in app.stream(inputs):
    for key, value in output.items():
        print(f"Finished running: {key}")
documents = value["documents"]
sources = ",".join([document.metadata["source"] for document in documents])
titles = ",".join([document.metadata["title"] for document in documents])
print("\n\n---")
print("generation: " + value["generation"])
print("sources: " + sources)
print("titles: " + titles)

1. DOCS RETRIEVE
Finished running: docs_retrieval
2. RELEVANCE CHECKER
3. DECIDE TO GENERATE
  GO TO GENERATE
Finished running: relevance_checker
4. GENERATE ANSWER
5. HALLUCINATION CHECKER
  NO HALLUCINATION
Finished running: generate_answer


---
generation: A prompt is a method used in prompt engineering, also known as In-Context Prompting, to communicate with language models (LLMs) to influence their behavior and achieve desired outcomes without altering the model's weights. This process involves experimentation and heuristics, as the effectiveness of prompts can vary significantly across different models. The primary goal of prompt engineering is to enhance alignment and steerability of the models.
sources: https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/,https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/,https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/,https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/
titles: Prompt